In [9]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from loader_util.preprocessing import (
    ImageToArrayPreprocessor,
    SimplePreprocessor,
    MeanPreprocessor,
    CropPreprocessor
)
from loader_util.io import HDF5DatasetGenerator
from loader_util.utils import rankn_accuracy
from tensorflow.keras.models import load_model, Model
import numpy as np
from tqdm import tqdm
import json

In [11]:
# script constants
dataset_mean_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/output/kaggle_cats_dogs.json"
saved_model_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/output/alexnet_cats_dogs_model.keras"
test_data_path = r"/content/drive/MyDrive/Colab Notebooks/ImageDataset/book2/kaggle_dogs_vs_cats/hdf5/test.hdf5"
BATCH_SIZE = 64

In [12]:
means = json.loads(open(dataset_mean_path).read())
sp = SimplePreprocessor(227, 227)
mp = MeanPreprocessor(means["r"], means["g"], means["b"])
cp = CropPreprocessor(227, 227)
iap = ImageToArrayPreprocessor()

print(f"[INFO] loading model......")
model = load_model(saved_model_path)
model: Model

[INFO] loading model......


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 32 variables whereas the saved optimizer has 62 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [13]:
print(f"[INFO] predicting on test data (no additional crops)......")
test_gen = HDF5DatasetGenerator(dbpath=test_data_path,
                                batch_size=BATCH_SIZE,
                                preprocessors=[sp, mp, iap])

simple_predictions = model.predict(test_gen.generator(),
                                   steps=test_gen.num_images // BATCH_SIZE)
rank1_acc, rank5_acc = rankn_accuracy(simple_predictions, test_gen.db["labels"])
print(f"[INFO] rank1_acc: {rank1_acc: 0.2f}")
test_gen.close()

[INFO] predicting on test data (no additional crops)......
[INFO] initialised HDF5 generator with 2500 data across 2 classes
39/39 ━━━━━━━━━━━━━━━━━━━━ 9s 220ms/step
[INFO] rank1_acc:  0.93


In [26]:
print(f"[INFO] predicting on test data (with crops now)......")
test_gen = HDF5DatasetGenerator(dbpath=test_data_path,
                                batch_size=BATCH_SIZE,
                                preprocessors=[mp])
crop_predictions = []

[INFO] predicting on test data (with crops now)......
[INFO] initialised HDF5 generator with 2500 data across 2 classes


In [27]:
# loop over a single pass
for i, (batch_images, batch_labels) in enumerate(test_gen.generator(passes=1)):
    for image in batch_images:
        crops = cp.preprocess(image)
        crops = np.array([iap.preprocess(cp)
                          for cp in crops], dtype="float32")

        crop_prediction = model.predict(crops, verbose=0)
        crop_predictions.append(crop_prediction.mean(axis=0))
    print(f"completed with batch : {i + 1}")

completed with batch : 1
completed with batch : 2
completed with batch : 3
completed with batch : 4
completed with batch : 5
completed with batch : 6
completed with batch : 7
completed with batch : 8
completed with batch : 9
completed with batch : 10
completed with batch : 11
completed with batch : 12
completed with batch : 13
completed with batch : 14
completed with batch : 15
completed with batch : 16
completed with batch : 17
completed with batch : 18
completed with batch : 19
completed with batch : 20
completed with batch : 21
completed with batch : 22
completed with batch : 23
completed with batch : 24
completed with batch : 25
completed with batch : 26
completed with batch : 27
completed with batch : 28
completed with batch : 29
completed with batch : 30
completed with batch : 31
completed with batch : 32
completed with batch : 33
completed with batch : 34
completed with batch : 35
completed with batch : 36
completed with batch : 37
completed with batch : 38
completed with batch 

In [28]:
len(crop_predictions)

2500

In [30]:
rank1_acc, rank5_acc = rankn_accuracy(np.array(crop_predictions), test_gen.db["labels"])
print(f"[INFO] rank1_acc: {rank1_acc: 0.2f}")
test_gen.close()

[INFO] rank1_acc:  0.93
